In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import datetime

import tensorflow as tf
import tfimm

import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy

In [2]:
# Create the base model
model_name = "vit_large_patch16_224"
base_model = tfimm.create_model(
    model_name,
    nb_classes=0  # this removes the final layer
)

# Load base_model weights (by_name=True because nb_classes=0 removes a layer)
model_path = "RETFound_CFP_weights.h5" # get from RETFound github

base_model.load_weights(model_path, by_name=True, skip_mismatch=False)
base_model.trainable = False
# base_model.summary()

# Input layer
input_shape = (224, 224, 3)
inputs = tf.keras.Input(shape=input_shape)

# ViT layer
x = base_model(inputs)

# Add layers to match the original architecture leading up to the 'head' layer
# x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
prediction = tf.keras.layers.Dense(1, activation='relu',name='head')(x)


# Define the model
model = tf.keras.Model(inputs=inputs, outputs=prediction)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vit_large_patch16_224 (ViT)  (None, 1024)             303301632 
                                                                 
 dense (Dense)               (None, 512)               524800    
                                                                 
 head (Dense)                (None, 1)                 513       
                                                                 
Total params: 303,826,945
Trainable params: 525,313
Non-trainable params: 303,301,632
_________________________________________________________________


In [5]:
def get_first_image_from_each_directory(root_dir):
    image_paths = []

    for class_dir in os.listdir(root_dir):
        full_class_dir = os.path.join(root_dir, class_dir)
        if os.path.isdir(full_class_dir):
            first_image = sorted(os.listdir(full_class_dir))[0]
            image_path = os.path.join(full_class_dir, first_image)
            image_paths.append(image_path)

    return image_paths

def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    left_half = tf.image.crop_to_bounding_box(image, offset_height=0, offset_width=0, target_height=512, target_width=512)
    image = tf.image.resize(left_half, [224, 224])  # Resize to a fixed size, e.g., 224x224
    image = image / 255.0  # Normalize to [0,1] range
    return image

def create_dataset(image_paths):
    image_paths = tf.convert_to_tensor(image_paths)

    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(lambda path: load_and_preprocess_image(path))
    return dataset

# Define the root directory of your dataset
root_dir = "./data/OIMHS dataset/Images"

# Get the first image from each directory
image_paths = get_first_image_from_each_directory(root_dir)

# Create the dataset
image_dataset = create_dataset(image_paths)

In [6]:
df = pd.read_excel('./data/OIMHS dataset/Demographics of the participants.xlsx')
df["Age(years)"]
labels = df["Age(years)"]
labels_dataset = tf.data.Dataset.from_tensor_slices(labels)
full_dataset = tf.data.Dataset.zip((image_dataset,labels_dataset))
# full_dataset = full_dataset.map(lambda x,y: {"image":x,"label":y})

In [7]:
full_dataset= full_dataset.batch(1)

In [9]:
full_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [8]:
for data in full_dataset.take(1):
    print(data)

(<tf.Tensor: shape=(1, 224, 224, 3), dtype=float32, numpy=
array([[[[0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079],
         ...,
         [0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079]],

        [[0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079],
         ...,
         [0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079]],

        [[0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079],
         ...,
         [0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079],
         [0.10196079, 0.10196079, 0.10196079]],

        ...,

        [[0.13945661, 0.13945661,

In [6]:
train_dataset = full_dataset.take(int(0.8 * len(full_dataset)))  # Take 80% of the data for training
validation_dataset = full_dataset.skip(int(0.8 * len(full_dataset)))

In [7]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
# Create the base model
model_name = "vit_large_patch16_224"
base_model = tfimm.create_model(
    model_name,
    nb_classes=0  # this removes the final layer
)

# Load base_model weights (by_name=True because nb_classes=0 removes a layer)
model_path = "RETFound_oct_weights.h5"

base_model.load_weights(model_path, by_name=True, skip_mismatch=False)
base_model.trainable = False
# base_model.summary()

# Input layer
input_shape = (224, 224, 3)
inputs = tf.keras.Input(shape=input_shape)

# ViT layer
x = base_model(inputs)

# Add layers to match the original architecture leading up to the 'head' layer
# x = tf.keras.layers.LayerNormalization()(x)
# x = tf.keras.layers.Dense(1024, activation='relu')(x)
prediction = tf.keras.layers.Dense(1, activation='relu',name='head')(x)


# Define the model
model = tf.keras.Model(inputs=inputs, outputs=prediction)
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vit_large_patch16_224 (ViT)  (None, 1024)             303301632 
                                                                 
 head (Dense)                (None, 1)                 1025      
                                                                 
Total params: 303,302,657
Trainable params: 1,025
Non-trainable params: 303,301,632
_________________________________________________________________


In [34]:
# Create the base model
model_name = "vit_large_patch16_224"
base_model = tfimm.create_model(
    model_name,
    nb_classes=0  # this removes the final layer
)

# Load base_model weights (by_name=True because nb_classes=0 removes a layer)
model_path = "RETFound_oct_weights.h5"

base_model.load_weights(model_path, by_name=True, skip_mismatch=False)
base_model.trainable = True
base_model.summary()

# Input layer
input_shape = (224, 224, 3)
inputs = tf.keras.Input(shape=input_shape)



# ViT layer
x = base_model(inputs)

# Add layers to match the original architecture leading up to the 'head' layer
# x = tf.keras.layers.LayerNormalization()(x)
# x = tf.keras.layers.Dense(1024, activation='relu')(x)
prediction = tf.keras.layers.Dense(1, activation='relu',name='head')(x)


# Define the model
model = tf.keras.Model(inputs=inputs, outputs=prediction)
model.summary()

ResourceExhaustedError: Exception encountered when calling layer "mlp" (type MLP).

failed to allocate memory [Op:AddV2]

Call arguments received:
  • x=tf.Tensor(shape=(1, 197, 1024), dtype=float32)
  • training=False

In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mean_absolute_error'])

In [12]:
%load_ext tensorboard
# launch from console with command: tensorboard --logdir=./

In [13]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

epochs=50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[tensorboard_callback]
)

Epoch 1/50
100/100 [==============================] - 54s 415ms/step - loss: 37.2498 - mean_absolute_error: 37.2498 - val_loss: 15.3315 - val_mean_absolute_error: 15.3315
Epoch 2/50
100/100 [==============================] - 38s 380ms/step - loss: 7.9966 - mean_absolute_error: 7.9966 - val_loss: 8.5096 - val_mean_absolute_error: 8.5096
Epoch 3/50
100/100 [==============================] - 38s 375ms/step - loss: 7.4488 - mean_absolute_error: 7.4488 - val_loss: 8.9324 - val_mean_absolute_error: 8.9324
Epoch 4/50
100/100 [==============================] - 38s 377ms/step - loss: 7.2446 - mean_absolute_error: 7.2446 - val_loss: 8.5424 - val_mean_absolute_error: 8.5424
Epoch 5/50
100/100 [==============================] - 38s 376ms/step - loss: 7.1487 - mean_absolute_error: 7.1487 - val_loss: 8.4421 - val_mean_absolute_error: 8.4421
Epoch 6/50
100/100 [==============================] - 38s 378ms/step - loss: 7.1648 - mean_absolute_error: 7.1648 - val_loss: 8.3790 - val_mean_absolute_error: 8

In [14]:
predictions = model.predict(test_dataset)
predictions

array([[67.34243 ],
       [69.85764 ],
       [69.15124 ],
       [68.004135],
       [69.25767 ]], dtype=float32)

In [15]:
y_true = []
for images, labels in test_dataset.take(6):
    y_true.append(tf.cast(labels, tf.int32))
    print(labels)

tf.Tensor([74], shape=(1,), dtype=int64)
tf.Tensor([59], shape=(1,), dtype=int64)
tf.Tensor([51], shape=(1,), dtype=int64)
tf.Tensor([51], shape=(1,), dtype=int64)
tf.Tensor([78], shape=(1,), dtype=int64)


In [16]:
y_true[0].numpy() - predictions[0]

array([6.65756989])

In [17]:
sumdiff=0
for i in range(len(y_true)):
    absdiff = np.abs(y_true[i].numpy() - predictions[i])
    sumdiff+=absdiff
mean_diff = sumdiff/(len(y_true))

In [18]:
mean_diff


array([12.28258362])